In [1]:
# Import some useful libraries

import pandas as pd
import numpy as np

In [2]:
# Import some more libraries that will help retrieval from the web using urls

import urllib
from urllib.request import urlretrieve

# urls for time series and area code data
url1 = "https://download.bls.gov/pub/time.series/la/la.data.60.Metro"
url2 = "https://download.bls.gov/pub/time.series/la/la.area"

In [3]:
# Save the data as .csv

urlretrieve(url1, '/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/timeseries.csv')
urlretrieve(url2, '/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/area_codes.csv')

('/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/area_codes.csv',
 <http.client.HTTPMessage at 0x7f93cf085910>)

In [4]:
# Read in time series data

df1 = pd.read_csv('/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/timeseries.csv', delimiter="\t")

/Users/toluayangbayi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# Check the first five rows

df1.head()

,series_id,year,period,value,footnote_codes
0,LASMT261982000000003,1990,M01,7.6,NaN
1,LASMT261982000000003,1990,M02,7.6,NaN
2,LASMT261982000000003,1990,M03,7.5,NaN
3,LASMT261982000000003,1990,M04,7.6,NaN
4,LASMT261982000000003,1990,M05,7.6,NaN


In [6]:
# Check the last five rows

df1.tail()

,series_id,year,period,value,footnote_codes
655667,LAUMT724198000000006,2021,M05,733777,NaN
655668,LAUMT724198000000006,2021,M06,723130,NaN
655669,LAUMT724198000000006,2021,M07,721640,NaN
655670,LAUMT724198000000006,2021,M08,730164,NaN
655671,LAUMT724198000000006,2021,M09,733179,P


In [7]:
# For some general info about the data
 
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655672 entries, 0 to 655671
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   series_id                       655672 non-null  object
 1   year                            655672 non-null  int64 
 2   period                          655672 non-null  object
 3          value                    655672 non-null  object
 4   footnote_codes                  1716 non-null    object
dtypes: int64(1), object(4)
memory usage: 25.0+ MB


In [8]:
# Looking at the info, I could see leading whitespace around the column called value so I checked the column names
# Doing this shows leading and trailing whitespaces around column names

df1.columns

Index(['series_id                     ', 'year', 'period', '       value',
       'footnote_codes'],
      dtype='object')

In [9]:
# Rename columns to clear the whitespace
# value is the indicator name in this case, I am aiming for the unemployment rate

df1.rename(columns={'series_id                     ':'series_id', '       value':'unemployment'}, inplace=True)

In [10]:
# Drop the column containing footnote codes

df1.drop('footnote_codes', axis = 1, inplace=True)

In [11]:
# Create a variable measure from series_id. I read from the metadata file that the last two characters of series_id 
# represent the name of an indicator. In this case 03 = unemployment rate. 
# See https://download.bls.gov/pub/time.series/la/la.measure for others

df1['measure'] = df1['series_id'].apply(lambda x: x[18:].strip())

In [12]:
# Since cleaning is an iterative process, I check the data again
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655672 entries, 0 to 655671
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   series_id     655672 non-null  object
 1   year          655672 non-null  int64 
 2   period        655672 non-null  object
 3   unemployment  655672 non-null  object
 4   measure       655672 non-null  object
dtypes: int64(1), object(4)
memory usage: 25.0+ MB


In [13]:
df1.head()

,series_id,year,period,unemployment,measure
0,LASMT261982000000003,1990,M01,7.6,03
1,LASMT261982000000003,1990,M02,7.6,03
2,LASMT261982000000003,1990,M03,7.5,03
3,LASMT261982000000003,1990,M04,7.6,03
4,LASMT261982000000003,1990,M05,7.6,03


In [14]:
# What other data values are in the measure variable?

df1['measure'].value_counts()

03    163918
04    163918
05    163918
06    163918
Name: measure, dtype: int64

In [15]:
# Filter unemployment data

df1 = df1[df1['measure'] == '03']
df1.reset_index(drop = True, inplace=True)

In [16]:
df1.tail()

,series_id,year,period,unemployment,measure
163913,LAUMT724198000000003,2021,M05,7.4,03
163914,LAUMT724198000000003,2021,M06,7.5,03
163915,LAUMT724198000000003,2021,M07,8.4,03
163916,LAUMT724198000000003,2021,M08,8.3,03
163917,LAUMT724198000000003,2021,M09,7.2,03


In [17]:
# Need to convert unemployment values to float. Apparently some values were "bad" and cannot be changed by 
# directly applying float()
# Build a dictionary of string unemployment values

cnt = 0

unemployment_dict = {}

for row in df1['unemployment']:
  if row is not None:
    try:
      float(row)
      pass
    except ValueError:
      unemployment_dict[cnt] = row
  cnt+=1

print(unemployment_dict)
print(len(unemployment_dict))

{161426: '           -', 161427: '           -', 161436: '           -', 161838: '           -', 161839: '           -', 161848: '           -', 162250: '           -', 162251: '           -', 162260: '           -', 162662: '           -', 162663: '           -', 162672: '           -', 163074: '           -', 163075: '           -', 163084: '           -', 163486: '           -', 163487: '           -', 163496: '           -', 163898: '           -', 163899: '           -', 163908: '           -'}
21


In [18]:
# check 
# df1.loc[161427]

In [19]:
# Remove bad characters by first naming them 'unavailable'. 
# Whitespace stripping did not work, but changing the values to helps to identify the bad data

for i in unemployment_dict:
  unemployment_dict[i] = "unavailable"
      
print(unemployment_dict)

{161426: 'unavailable', 161427: 'unavailable', 161436: 'unavailable', 161838: 'unavailable', 161839: 'unavailable', 161848: 'unavailable', 162250: 'unavailable', 162251: 'unavailable', 162260: 'unavailable', 162662: 'unavailable', 162663: 'unavailable', 162672: 'unavailable', 163074: 'unavailable', 163075: 'unavailable', 163084: 'unavailable', 163486: 'unavailable', 163487: 'unavailable', 163496: 'unavailable', 163898: 'unavailable', 163899: 'unavailable', 163908: 'unavailable'}


In [20]:
## Replace values in main dataframe
for i in unemployment_dict:
  df1.loc[i, 'unemployment'] = unemployment_dict[i]

In [21]:
df2 = df1[df1['unemployment'] != 'unavailable']

In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163897 entries, 0 to 163917
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   series_id     163897 non-null  object
 1   year          163897 non-null  int64 
 2   period        163897 non-null  object
 3   unemployment  163897 non-null  object
 4   measure       163897 non-null  object
dtypes: int64(1), object(4)
memory usage: 7.5+ MB


In [23]:
# Having removed the 'bad' values, the 'good' ones can now be converted to float data

df2['unemployment'] = df2['unemployment'].apply(float)

<ipython-input-23-cb4848acbc69>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['unemployment'] = df2['unemployment'].apply(float)


In [24]:
# To keep the data small I filter 2020 to date

# df3 = df2.loc[df2['year'] >= 2020]
# df3.info()

In [25]:
# Since I need to know the names of the MSAs, I create a new column 'area_code' using appropriate indexes
# from series_id
# This new column will be used to create a join with the data file containing names of MSAs

df2["area_code"] = df2['series_id'].apply(lambda x: x[3:18])

<ipython-input-25-c5929f5651c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["area_code"] = df2['series_id'].apply(lambda x: x[3:18])


In [26]:
df2.head()

,series_id,year,period,unemployment,measure,area_code
0,LASMT261982000000003,1990,M01,7.6,03,MT2619820000000
1,LASMT261982000000003,1990,M02,7.6,03,MT2619820000000
2,LASMT261982000000003,1990,M03,7.5,03,MT2619820000000
3,LASMT261982000000003,1990,M04,7.6,03,MT2619820000000
4,LASMT261982000000003,1990,M05,7.6,03,MT2619820000000


In [27]:
# df2 (unemployment values dataset) will be merged with df3 (area code names) to get 
# a panel dataset of unemployment rate in metropolitan statistical areas

In [28]:
# Read in the area codes data. The data is tab delimited
# This data will be mapped to the unemployment time series

df3 = pd.read_csv('/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/area_codes.csv', delimiter="\t")
df3.head(5)

,area_type_code,area_code,area_text,display_level,selectable,sort_sequence
0,A,ST0100000000000,Alabama,0,T,1
1,A,ST0200000000000,Alaska,0,T,146
2,A,ST0400000000000,Arizona,0,T,188
3,A,ST0500000000000,Arkansas,0,T,252
4,A,ST0600000000000,California,0,T,378


In [29]:
# Check for issues with column names

df3.columns

Index(['area_type_code', 'area_code', 'area_text', 'display_level',
       'selectable', 'sort_sequence'],
      dtype='object')

In [30]:
# Will use area code B for metropolitan statistical areas
# See https://download.bls.gov/pub/time.series/la/la.area_type

df3['area_type_code'].value_counts()

F    3222
G    1879
H    1430
D     562
B     396
I     313
E     178
K     160
J      71
A      52
C      38
N       9
L       7
M       4
Name: area_type_code, dtype: int64

In [31]:
# Filter metropolitan areas
df4 = df3[df3['area_type_code'] == 'B']

In [32]:
df4['area_type_code'].value_counts()

B    396
Name: area_type_code, dtype: int64

In [33]:
# Merge unemployment statistics with MSA names

df5 = df2.merge(df4, on='area_code')

In [34]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163897 entries, 0 to 163896
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   series_id       163897 non-null  object 
 1   year            163897 non-null  int64  
 2   period          163897 non-null  object 
 3   unemployment    163897 non-null  float64
 4   measure         163897 non-null  object 
 5   area_code       163897 non-null  object 
 6   area_type_code  163897 non-null  object 
 7   area_text       163897 non-null  object 
 8   display_level   163897 non-null  int64  
 9   selectable      163897 non-null  object 
 10  sort_sequence   163897 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 15.0+ MB


In [35]:
df5.head()

,series_id,year,period,unemployment,measure,area_code,area_type_code,area_text,display_level,selectable,sort_sequence
0,LASMT261982000000003,1990,M01,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
1,LASMT261982000000003,1990,M02,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
2,LASMT261982000000003,1990,M03,7.5,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
3,LASMT261982000000003,1990,M04,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
4,LASMT261982000000003,1990,M05,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733


In [36]:
# Check the periodicity of the data using the 'period' column
# There are 13 values where M13 is the annual data.

df5.period.value_counts()

M01    12736
M02    12736
M05    12736
M06    12736
M07    12736
M08    12736
M09    12736
M03    12729
M04    12729
M10    12342
M11    12338
M12    12338
M13    12269
Name: period, dtype: int64

In [37]:
# Rows where the period value is M13 can be selected for analysis with annual data

annual_unemployment = df5[df5['period'] == 'M13']
annual_unemployment.head()

,series_id,year,period,unemployment,measure,area_code,area_type_code,area_text,display_level,selectable,sort_sequence
394,LAUMT261982000000003,1990,M13,7.9,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
407,LAUMT261982000000003,1991,M13,9.8,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
420,LAUMT261982000000003,1992,M13,9.5,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
433,LAUMT261982000000003,1993,M13,7.3,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
446,LAUMT261982000000003,1994,M13,5.7,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733


In [38]:
# But I am interested in the monthly data so I filtered M13 rows

monthly_unemployment = df5[df5['period'] != 'M13']
monthly_unemployment.head()

,series_id,year,period,unemployment,measure,area_code,area_type_code,area_text,display_level,selectable,sort_sequence
0,LASMT261982000000003,1990,M01,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
1,LASMT261982000000003,1990,M02,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
2,LASMT261982000000003,1990,M03,7.5,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
3,LASMT261982000000003,1990,M04,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733
4,LASMT261982000000003,1990,M05,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati...",0,T,3733


In [39]:
# I save the monthly data to csv
monthly_unemployment.to_csv("/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/monthly_unemployment.csv", index=False)

In [40]:
# This can be read in during subsequent analysis
# monthly_unemployment = pd.read_csv("/Users/toluayangbayi/Documents/DataScience/Using-LAUS-flatfiles/monthly_unemployment.csv")

In [41]:
# Some further data cleaning by dropping out unneeded columns of data

monthly_unemploymentdf = monthly_unemployment.drop(['series_id', 'display_level', 'selectable', 'sort_sequence'], axis = 1)

In [42]:
# Finally I have a clean dataframe of monthly unemployment data for 396 U.S. metropolitan statistical areas
# Of course some further cleaning can be done for instance removing 'Metropolitan Statistical Area' from 
# 'area_txt' column to produce the city-state names of the MSA only. A column of city or states only can be produced too.

monthly_unemploymentdf.head()

,year,period,unemployment,measure,area_code,area_type_code,area_text
0,1990,M01,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati..."
1,1990,M02,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati..."
2,1990,M03,7.5,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati..."
3,1990,M04,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati..."
4,1990,M05,7.6,03,MT2619820000000,B,"Detroit-Warren-Dearborn, MI Metropolitan Stati..."
